In [1]:
 !git clone https://github.com/htrnguyen/vietocr.git

Cloning into 'vietocr'...
remote: Enumerating objects: 3301, done.
remote: Counting objects: 100% (805/805), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 3301 (delta 729), reused 691 (delta 673), pack-reused 2496 (from 2)
Receiving objects: 100% (3301/3301), 3.83 MiB | 37.35 MiB/s, done.
Resolving deltas: 100% (2445/2445), done.


In [2]:
import sys, os, time
import torch
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

sys.path.append('/kaggle/working/vietocr')
from vietocr.tool.config import Cfg
from vietocr.tool.predictor import Predictor

# Đảm bảo dùng GPU nếu có
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Đường dẫn dữ liệu và model
DATA_ROOT = '/kaggle/input/nckh-2425-crops'
CSV_ANN = '/kaggle/input/nckh-2425-crops/crops_gt.csv'
WEIGHT_PATH = '/kaggle/input/vietocr_fineturning/gguf/default/1/transformerocr.pth'

Device: cuda:0


In [3]:
def extract_vocab_from_labels(label_files):
    chars = set()
    for fpath in label_files:
        with open(fpath, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    _, label = line.strip().split("\t")
                    chars.update(label)
                except:
                    continue
    return "".join(sorted(chars))

vocab = extract_vocab_from_labels([
    '/kaggle/input/vietocr-data-pretrain/dataset/test_annotation.txt',
    '/kaggle/input/vietocr-data-pretrain/dataset/train_annotation.txt'
])

In [4]:
config = Cfg.load_config_from_name("vgg_transformer")
config["vocab"] = vocab
config["weights"] = WEIGHT_PATH
config["device"] = device
predictor = Predictor(config)
config

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:30<00:00, 18.8MB/s] 
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


{'vocab': " !%&'(),-./0123456789:ABCDEFGHIJKLMNOPQRSTUVWXYZ\\abcdefghijklmnopqrstuvwxyzÀÁÂÈÉÊÌÍÒÓÔÙÚÝàáâãèéêìíòóôùúĂăĐđĨŨũŪƠơƯưẠạẢảẤấẦầẨẩẫẬậẮắẰằẵẶặẸẹẺẻẾếỀềỂểỄỆệỉỊịỌọỎỏỐốỒồỔổỖỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỸ",
 'device': 'cuda:0',
 'seq_modeling': 'transformer',
 'transformer': {'d_model': 256,
  'nhead': 8,
  'num_encoder_layers': 6,
  'num_decoder_layers': 6,
  'dim_feedforward': 2048,
  'max_seq_length': 1024,
  'pos_dropout': 0.1,
  'trans_dropout': 0.1},
 'optimizer': {'max_lr': 0.0003, 'pct_start': 0.1},
 'trainer': {'batch_size': 32,
  'print_every': 200,
  'valid_every': 4000,
  'iters': 100000,
  'export': './weights/transformerocr.pth',
  'checkpoint': './checkpoint/transformerocr_checkpoint.pth',
  'log': './train.log',
  'metrics': None},
 'dataset': {'name': 'data',
  'data_root': './img/',
  'train_annotation': 'annotation_train.txt',
  'valid_annotation': 'annotation_val_small.txt',
  'image_height': 32,
  'image_min_width': 32,
  'image_max_width': 512},
 'dataloader': {'num

In [5]:
!git clone https://github.com/htrnguyen/compare_ocr_benchmark.git

Cloning into 'compare_ocr_benchmark'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 72 (delta 30), reused 63 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 4.09 MiB | 6.12 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [6]:
!pip install python-Levenshtein jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.8 MB/s eta 0:00:00:00:01


In [7]:
import sys
sys.path.append('/kaggle/working/compare_ocr_benchmark/common')
from metrics import compute_metrics
from utils import save_results

In [8]:
df = pd.read_csv(CSV_ANN)

results = []
for idx, row in df.iterrows():
    fname = row['filename']
    desc_gt = row['description_gt']
    label = row.get('label', '')
    img_path = os.path.join(DATA_ROOT, fname)

    try:
        image = Image.open(img_path).convert("RGB")
        t1 = time.perf_counter()
        pred = predictor.predict(image)
        t2 = time.perf_counter()
        infer_time = round(t2 - t1, 3)
    except Exception as e:
        pred = f"OCR_Error: {e}"
        infer_time = 0.0

    # Dùng hàm metrics chung (import từ compare_ocr_benchmark/common/metrics.py)
    from metrics import compute_metrics
    metrics = compute_metrics(desc_gt, pred)

    results.append({
        "filename": fname,
        "label": label,
        "ground_truth": desc_gt,
        "predicted_text": pred,
        "cer": metrics["cer"],
        "wer": metrics["wer"],
        "lev": metrics["lev"],
        "acc": metrics["acc"],
        "time": infer_time
    })
    if idx % 50 == 0:
        print(f"Processed {idx}/{len(df)}")

Processed 0/2284
Processed 50/2284
Processed 100/2284
Processed 150/2284
Processed 200/2284
Processed 250/2284
Processed 300/2284
Processed 350/2284
Processed 400/2284
Processed 450/2284
Processed 500/2284
Processed 550/2284
Processed 600/2284
Processed 650/2284
Processed 700/2284
Processed 750/2284
Processed 800/2284
Processed 850/2284
Processed 900/2284
Processed 950/2284
Processed 1000/2284
Processed 1050/2284
Processed 1100/2284
Processed 1150/2284
Processed 1200/2284
Processed 1250/2284
Processed 1300/2284
Processed 1350/2284
Processed 1400/2284
Processed 1450/2284
Processed 1500/2284
Processed 1550/2284
Processed 1600/2284
Processed 1650/2284
Processed 1700/2284
Processed 1750/2284
Processed 1800/2284
Processed 1850/2284
Processed 1900/2284
Processed 1950/2284
Processed 2000/2284
Processed 2050/2284
Processed 2100/2284
Processed 2150/2284
Processed 2200/2284
Processed 2250/2284


In [9]:
import os
OUT_CSV = '/kaggle/working/compare_ocr_benchmark/results/vietocr_pretrain_results.csv'
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
save_results(results, OUT_CSV)

Lưu thành công: /kaggle/working/compare_ocr_benchmark/results/vietocr_pretrain_results.csv
